In [1]:
import os
import pandas as pd
import sys

In [43]:
WORKSHEETS_NAMES = ['Financial Data - Valuescience','Financial Data - Orbis','Financial Data - Factiva','Founder Data - Orbis',
                'Founder Data - Factiva','Funding Data - Quid', 'Patent Records (company)','Patent Families (company)', 'Patent Records (founder)',
                'Patent Families (company)', 'Sci. Lit. Data (company)', 'H-Index (company)', 'Sci. Lit. Data (founder)', 'H-Index (founder)']

In [3]:
""" helper for appending data frames to csv"""
def append_to_csv(df, path_to_csv, sep=";"):
    import os
    if not os.path.isfile(path_to_csv):
        df.to_csv(path_to_csv, mode='a', index=False, sep=sep)
    elif len(df.columns) != len(pd.read_csv(path_to_csv, nrows=1, sep=sep).columns):
        raise Exception("Columns do not match!! Dataframe has " + str(len(df.columns)) + " columns. CSV file has " + str(len(pd.read_csv(path_to_csv, nrows=1, sep=sep).columns)) + " columns.")
    elif not (df.columns == pd.read_csv(path_to_csv, nrows=1, sep=sep).columns).all():
        raise Exception("Columns and column order of dataframe and csv file do not match!!")
    else:
        df.to_csv(path_to_csv, mode='a', index=False, sep=sep, header=False)


In [4]:
"""get sheet with financial as dataframe
   do basic clean up: remove na"""
def get_sheet_financial(path_to_workbook, sheet_index, company, category, skip=0):
    # Read the excel sheet to pandas dataframe
    df = pd.read_excel(path_to_workbook, sheetname=sheet_index,skiprows=skip)

    # do basic reshaping/cleaning with non emtpy sheets
    if df.size > 0:
        cols = [0,2]
        df.drop(df.columns[cols], axis=1, inplace=True) # drop empty columns
        df.rename(columns=({'Period': 'KPI'}), inplace=True)
        df.drop([0, 1, 2, 3, 4], axis=0, inplace=True) # drop unused header rows
        #df.dropna(axis=0, how='all', inplace=True)

        # unpivot
        df = pd.melt(df, id_vars=['KPI'], var_name='Year', value_name='Value')
        df.dropna(axis=0, how='any', inplace=True)

        # add extra columns and swap
        df['Company'] = company
        df['Category'] = category
        cols = df.columns.tolist()
        cols = cols[-2:] + cols[:-2]
        df = df[cols]
        return df

In [5]:
""" consolidation for standard sheets where each column contains data for one kpi
    and one columns contains the year"""
def get_sheet(path_to_workbook, sheet_index, company, category, year_column, skip=0):
    df = pd.read_excel(path_to_workbook, sheetname=sheet_index,skiprows=skip)
    if df.size > 0:

        # unpivot
        df = pd.melt(df, id_vars=[year_column], var_name='KPI', value_name='Value')
        df.dropna(axis=0, how='any', inplace=True)

        # add extra columns and swap
        df['Company'] = company
        df['Category'] = category
        df.rename(columns=({year_column: 'Year'}), inplace=True)
        cols = df.columns.tolist()
        cols = cols[-2:] + cols[:-2]
        tmp = cols[2]
        cols[2] = cols[3]
        cols[3] = tmp
        df = df[cols]
        return df

In [45]:
# main
# first argument is path to dir with import Excel
# todo: usage statement if wrong number of arguments
if len(sys.argv) != 3:
    print('USAGE: python consolidate_excel {path_to_dir} {output_options[0=rows, 1=pivot]}')
    sys.exit()
path = '/Users/dirk/python' #sys.argv[1]
output_option = '1' #sys.argv[2]
# init/delete old output file
output_file = os.path.join(path, 'out.csv')
if os.path.isfile(output_file):
    os.remove(output_file)
all_dfs = None

# traverse all Excel files in given folder
for filename in os.listdir(path):
    if filename.endswith(".xls") or filename.endswith(".xlsx"):
        # get additional information from workbook
        xls = pd.ExcelFile(os.path.join(path, filename), on_demand = True)
        sheets = xls.sheet_names
        company = (filename.split('_'))[1]

        df = None
        for sh in sheets:
            df = None
            if sh == WORKSHEETS_NAMES[0]:
                #print('reading', sh)
                df = get_sheet_financial(os.path.join(path, filename), sheets.index(WORKSHEETS_NAMES[0]), company, WORKSHEETS_NAMES[0], 1)

            # Funding data - quid
            if sh == WORKSHEETS_NAMES[5]:
                #print('reading', sh)
                df = get_sheet(os.path.join(path, filename), sheets.index(WORKSHEETS_NAMES[5]), company, WORKSHEETS_NAMES[5], 'Year of Funding Event', 2)

            # patent
            if sh == WORKSHEETS_NAMES[6]:
                print('reading', sh)
                df = get_sheet(os.path.join(path, filename), sheets.index(WORKSHEETS_NAMES[6]), company, WORKSHEETS_NAMES[6], 'Publication Year', 1)
            
            # literature(founder)
            if sh == WORKSHEETS_NAMES[12]:
                print('reading', sh)
                df = get_sheet(os.path.join(path, filename), sheets.index(WORKSHEETS_NAMES[12]), company, WORKSHEETS_NAMES[12], 'PY', 0)


            if df is not None:
                #fname, file_extension = os.path.splitext(os.path.join(path, filename))
                #print(df.head())
                #df.drop_duplicates(inplace=True)
                #append_to_csv(df, output_file)
                all_dfs = pd.concat([all_dfs, df])

# output to csv
# todo add filter for year as input parameter and adapt export accordingly
if output_option == '1':
    
    #print(WORKSHEETS_NAMES[12])
    print('output as pivot...', output_option)
    piv_df = all_dfs.pivot_table(values='Value', index=['Company','Year'], columns='KPI', aggfunc='first')
    piv_df.to_csv(output_file, index=True, sep=';')
    
    #print(piv_df.describe())
    print('head(20):')
    print(piv_df.head(n=20))
else:
    print('output as rows...')
    all_dfs.to_csv(output_file, index=False, sep=';')
print('file written:', output_file)


reading Patent Records (company)
reading Sci. Lit. Data (founder)
output as pivot... 1
head(20):
KPI                                                                   AB  \
Company          Year                                                      
iCo Therapeutics 2007                                               None   
                 2008                                               None   
                 2009                                               None   
                 2011                                               None   
                 2013                                               None   
                 2015  Introduction: Electrical impedance myography (...   
                 2016                                               None   

KPI                                                                   AF  \
Company          Year                                                      
iCo Therapeutics 2007                                             

In [46]:
%quickref